# Introduction to SageMaker TensorFlow - Image Classification

In [ ]:
#from https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/main/introduction_to_amazon_algorithms/image_classification_tensorflow/Amazon_TensorFlow_Image_Classification.ipynb
#tutorial https://aws.amazon.com/blogs/machine-learning/classify-your-own-images-using-amazon-sagemaker/

---
Welcome to [Amazon SageMaker Built-in Algorithms](https://sagemaker.readthedocs.io/en/stable/algorithms/index.html)! You can use SageMaker Built-in algorithms to solve many Machine Learning tasks through [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html). You can also use these algorithms through one-click in SageMaker Studio via [JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html).

In this demo notebook, we demonstrate how to use the TensorFlow Image Classification algorithm. Image Classification refers to classifying an image to one of the class labels of the training dataset.  We demonstrate two use cases of TensorFlow Image Classification models:

* How to use a model pre-trained on ImageNet dataset to classify an image. [ImageNetLabels](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt).
* How to fine-tune a pre-trained model to a custom dataset, and then run inference on the fine-tuned model.

Note: This notebook was tested on ml.t3.medium instance in Amazon SageMaker Studio with Python 3 (Data Science) kernel and in Amazon SageMaker Notebook instance with conda_python3 kernel.

---

1. [Set Up](#1.-Set-Up)
2. [Select a pre-trained model](#2.-Select-a-pre-trained-model)
3. Tutorial section not needed and removed for simplicity
4. [Fine-tune the pre-trained model on a custom dataset](#4.-Fine-tune-the-pre-trained-model-on-a-custome-dataset)
    * [Retrieve Training artifacts](#4.1.-Retrieve-Training-artifacts)
    * [Set Training parameters](#4.2.-Set-Training-parameters)
    * [Train with Automatic Model Tuning (HPO)](#AMT)
    * [Start Training](#4.4.-Start-Training)
    * [Extract Training performance metrics](#4.5.-Extract-Training-performance-metrics)
    * [Deploy & run Inference on the fine-tuned model](#4.6.-Deploy-&-run-Inference-on-the-fine-tuned-model)
    * [Incrementally train the fine-tuned model](#4.7.-Incrementally-train-the-fine-tuned-model)

## 1. Set Up
***
Before executing the notebook, there are some initial steps required for setup. This notebook requires latest version of sagemaker and ipywidgets.
***

In [2]:
!pip install sagemaker ipywidgets --upgrade --quiet


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


---

To train and host on Amazon Sagemaker, we need to setup and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook instance as the AWS account role with SageMaker access. It has necessary permissions, including access to your data in S3. 

---

In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. Select a pre-trained model
***
You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).
***

In [4]:
#model_id, model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"
model_id, model_version ="tensorflow-ic-resnet-50-classification-1", "*"
#model_id, model_version ="tensorflow-ic-efficientnet-b0-classification-1", "*"
#model_id, model_version ="tensorflow-ic-tf2-preview-inception-v3-classification-4", "*"


***
[Optional] Select a different Sagemaker pre-trained model. Here, we download the model_manifest file from the Built-In Algorithms s3 bucket, filter-out all the Image Classification models and select a model for inference.
***

In [5]:
import IPython
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models
from sagemaker.jumpstart.filters import And

# Retrieves all TensorFlow Image Classification models made available by SageMaker Built-In Algorithms.
filter_value = And("task == ic", "framework == tensorflow")
ic_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown, for user to select a model.
dropdown = Dropdown(
    options=ic_models,
    value=model_id,
    description="SageMaker Built-In TensorFlow Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("## Select a SageMaker pre-trained model from the dropdown below"))
display(dropdown)

## Select a SageMaker pre-trained model from the dropdown below

Dropdown(description='SageMaker Built-In TensorFlow Image Classification Models:', index=57, layout=Layout(wid…

In [6]:
ic_models

['tensorflow-ic-bit-m-r101x1-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-m-r101x1-imagenet21k-classification-1',
 'tensorflow-ic-bit-m-r101x3-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-m-r101x3-imagenet21k-classification-1',
 'tensorflow-ic-bit-m-r50x1-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-m-r50x1-imagenet21k-classification-1',
 'tensorflow-ic-bit-m-r50x3-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-m-r50x3-imagenet21k-classification-1',
 'tensorflow-ic-bit-s-r101x1-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-s-r101x3-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-s-r50x1-ilsvrc2012-classification-1',
 'tensorflow-ic-bit-s-r50x3-ilsvrc2012-classification-1',
 'tensorflow-ic-efficientnet-b0-classification-1',
 'tensorflow-ic-efficientnet-b1-classification-1',
 'tensorflow-ic-efficientnet-b2-classification-1',
 'tensorflow-ic-efficientnet-b3-classification-1',
 'tensorflow-ic-efficientnet-b4-classification-1',
 'tensorflow-ic-efficientnet-b5-classif

## 4. Fine-tune the pre-trained model on a custom dataset
***
Previously, we saw how to run inference on a pre-trained model. Next, we discuss how a model can be fine-tuned to a custom dataset with any number of classes. 

The model available for fine-tuning attaches a classification layer to the corresponding feature extractor model available on TensorFlow/PyTorch hub, and initializes the layer parameters to random values. The output dimension of the classification layer is determined based on the number of classes in the input data. The fine-tuning step fine-tunes the model parameters. The objective is to minimize classification error on the input data. The model returned by fine-tuning can be further deployed for inference. Below are the instructions for how the training data should be formatted for input to the model.

- **Input:** A directory with as many sub-directories as the number of classes. 
    - Each sub-directory should have images belonging to that class in .jpg format. 
- **Output:** A trained model that can be deployed for inference. 
    - A label mapping file is saved along with the trained model file on the s3 bucket.  
    
The input directory should look like below if the training data contains images from two classes: roses and dandelion. The s3 path should look like `s3://bucket_name/input_directory/`. Note the trailing `/` is required. The names of the folders and 'roses', 'dandelion', and the .jpg filenames can be anything. The label mapping file that is saved along with the trained model on the s3 bucket maps the folder names 'roses' and 'dandelion' to the indices in the list of class probabilities the model outputs. The mapping follows alphabetical ordering of the folder names. In the example below, index 0 in the model output list would correspond to 'dandelion' and index 1 would correspond to 'roses'.

    input_directory
        |--roses
            |--abc.jpg
            |--def.jpg
        |--dandelion
            |--ghi.jpg
            |--jkl.jpg

We provide tf_flowers dataset as a default dataset for fine-tuning the model. tf_flower comprises images of five types of flowers. The dataset has been downloaded from [TensorFlow](https://www.tensorflow.org/datasets/catalog/tf_flowers) under [Apache 2.0 License](https://jumpstart-cache-prod-us-west-2.s3-us-west-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt).
***

### 4.1. Retrieve Training artifacts
***
Here, for the selected model, we retrieve the training docker container, the training algorithm source, the pre-trained base model, and a python dictionary of the training hyper-parameters that the algorithm accepts with their default values. Note that the model_version="*" fetches the lates model. Also, we do need to specify the training_instance_type to fetch train_image_uri.
***

In [38]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = dropdown.value, "*"
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

In [39]:
model_id

'tensorflow-ic-resnet-50-classification-1'

### 4.2. Set Training parameters
***
Now that we are done with all the setup that is needed, we are ready to fine-tune our Image Classification model. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator will launch the training job. 

There are two kinds of parameters that need to be set for training. 

The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training. We defined the training instance type above to fetch the correct train_image_uri. 

The second set of parameters are algorithm specific training hyper-parameters.
***

In [40]:
# Sample training data is available in this bucket
#training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
#training_data_prefix = "training-datasets/tf_flowers/"

#training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"
#training_dataset_s3_path = f"s3://contrail-classification-input-101022"
#training_dataset_s3_path = f"s3://contrail-classification-input-101522"
training_dataset_s3_path = f"s3://contrail-classification-input-b4"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-ic-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

***
For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.
***

In [41]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
#hyperparameters["epochs"] = "5"
hyperparameters["epochs"] = "25"
print(hyperparameters)

{'train_only_top_layer': 'True', 'epochs': '25', 'batch_size': '32', 'optimizer': 'adam', 'learning_rate': '0.001', 'beta_1': '0.9', 'beta_2': '0.999', 'momentum': '0.9', 'epsilon': '1e-07', 'rho': '0.95', 'initial_accumulator_value': '0.1', 'reinitialize_top_layer': 'Auto', 'early_stopping': 'False', 'early_stopping_patience': '5', 'early_stopping_min_delta': '0.0', 'dropout_rate': '0.2', 'regularizers_l2': '0.0001', 'label_smoothing': '0.1', 'image_resize_interpolation': 'bilinear', 'augmentation': 'False', 'augmentation_random_flip': 'horizontal_and_vertical', 'augmentation_random_rotation': '0.2', 'augmentation_random_zoom': '0.1', 'binary_mode': 'False', 'eval_metric': 'accuracy', 'validation_split_ratio': '0.2'}


### 4.3. Train with Automatic Model Tuning ([HPO](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)) <a id='AMT'></a>
***
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We will use a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) object to interact with Amazon SageMaker hyperparameter tuning APIs.
***

In [42]:
from sagemaker.tuner import ContinuousParameter

# Use AMT for tuning and selecting the best model
use_amt = False 
#use_amt = True #RVM update 10/13 for better metric?

# Define objective metric per framework, based on which the best model will be selected.
amt_metric_definitions = {
    "metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
    "type": "Maximize",
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 2

### 4.4. Start Training
***
We start by creating the estimator object with all the required assets and then launch the training job.
***

In [43]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

training_metric_definitions = [
    {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "train_accuracy", "Regex": "- accuracy: ([0-9\\.]+)"},
    {"Name": "train_loss", "Regex": "- loss: ([0-9\\.]+)"},
]

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=training_metric_definitions,
)

if use_amt:

    hp_tuner = HyperparameterTuner(
        ic_estimator,
        amt_metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        amt_metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=amt_metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

2022-10-30 12:17:55 Starting - Starting the training job...
2022-10-30 12:18:22 Starting - Preparing the instances for trainingProfilerReport-1667132275: InProgress
.........
2022-10-30 12:19:49 Downloading - Downloading input data...
2022-10-30 12:20:20 Training - Downloading the training image.....................
2022-10-30 12:23:51 Training - Training image download completed. Training in progress..2022-10-30 12:23:55.096646: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-10-30 12:23:55.107682: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-10-30 12:23:55.456390: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-10-30 12:24:00,956 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
20

### 4.5. Extract Training performance metrics, note a regex issue gives an invalid result in Section 4.5
***
Performance metrics such as training accuracy/loss and validation accuracy/loss can be accessed through cloudwatch while the training. Code below provides the link to the cloudwatch log where these metrics can be found. 

Note that default resolution in Amazon Cloudwatch is one minute i.e. it averages the metrics logged within a single minute interval. Amazon CloudWatch also supports [high-resolution custom metrics](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/publishingMetrics.html), and its finest resolution is 1 second. However, the finer the resolution, the shorter the lifespan of the CloudWatch metrics. For the 1-second frequency resolution, the CloudWatch metrics are available for 3 hours. For more information about the resolution and the lifespan of the CloudWatch metrics, see [GetMetricStatistics](https://docs.aws.amazon.com/AmazonCloudWatch/latest/APIReference/API_GetMetricStatistics.html) in the Amazon CloudWatch API Reference.
***

In [49]:
if use_amt:
    training_job_name = hp_tuner.best_training_job()
else:
    training_job_name = ic_estimator.latest_training_job.job_name

In [46]:
import sagemaker
from IPython.core.display import Markdown

sagemaker_session = sagemaker.Session()

link = (
    "https://console.aws.amazon.com/cloudwatch/home?region="
    + sagemaker_session.boto_region_name
    + "#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20"
    + training_job_name
)
display(Markdown("CloudWatch metrics: [link](" + link + ")"))

CloudWatch metrics: [link](https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20jumpstart-example-tensorflow-ic-resnet--2022-10-30-12-17-55-060)

***
Alternatively, we can also fetch these metrics and analyze them within the notebook.
***

In [47]:
from sagemaker import TrainingJobAnalytics

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()

df.head(10)

,timestamp,metric_name,value
0,0.0,val_accuracy,3.83060
1,0.0,val_loss,0.85458
2,0.0,train_accuracy,0.78942
3,0.0,train_loss,0.85798


In [ ]:
training_job_name

***
We can filter out different metrics by names as well.
***

In [50]:
metric_names = [metric["Name"] for metric in training_metric_definitions]

metrics_df = {
    metric_name: df.query(f"metric_name == '{metric_name}'") for metric_name in metric_names
}

metrics_df["val_loss"]

,timestamp,metric_name,value
1,0.0,val_loss,0.85458


## 4.6. Deploy & run Inference on the fine-tuned model
***
A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the class label of an image. We follow the same steps as in the [Section 3 - Run inference on the pre-trained model](#3.-Run-inference-on-the-pre-trained-model). We start by retrieving the artifacts for deploying an endpoint. However, instead of base_predictor, we  deploy the `ic_estimator` that we fine-tuned.
***

In [17]:
#inference_instance_type = "ml.p2.xlarge"
#inference_instance_type = "ml.p3.xlarge"
inference_instance_type = "ml.m5.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = (hp_tuner if use_amt else ic_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

----!

In [18]:
model_id

'tensorflow-ic-resnet-50-classification-1'

In [34]:
#Load test data in a list
import os
#jpg_files = [f for f in os.listdir('/root/contrail_processed/Neg') if f.endswith('.JPG')]
#jpg_files = [f for f in os.listdir('/root/contrail_processed/Neg/filename00012_archive') if f.endswith('B4.JPG')]
jpg_files = [f for f in os.listdir('/root/contrail_processed/Pos/') if f.endswith('B4.JPG')]


In [35]:
jpg_files

['LC08_L1TP_008014_20180402_20180416_01_T1_B4.JPG',
 'LC08_L1TP_032035_20180410_20180417_01_T1_B4.JPG',
 'LC08_L1TP_032042_20180120_20180206_01_T1_B4.JPG',
 'LC08_L1TP_021048_20181225_20190129_01_T1_B4.JPG',
 'LC08_L1TP_223069_20180927_20181009_01_T1_B4.JPG',
 'LC08_L1TP_026015_20180907_20180912_01_T1_B4.JPG',
 'LC08_L1TP_020036_20180812_20180828_01_T1_B4.JPG',
 'LC08_L1TP_004072_20181015_20181030_01_T1_B4.JPG',
 'LC08_L1TP_045030_20180320_20180403_01_T1_B4.JPG',
 'LC08_L1TP_079011_20180825_20180829_01_T1_B4.JPG',
 'LC08_L1TP_041024_20180815_20180829_01_T1_B4.JPG',
 'LC08_L1TP_019045_20180210_20180222_01_T1_B4.JPG',
 'LC08_L1TP_008026_20181011_20181030_01_T1_B4.JPG',
 'LC08_L1TP_005028_20180803_20180814_01_T1_B4.JPG',
 'LC08_L1TP_219074_20180611_20180615_01_T1_B4.JPG',
 'LC08_L1TP_061012_20180710_20180717_01_T1_B4.JPG',
 'LC08_L1TP_012031_20180905_20180912_01_T1_B4.JPG',
 'LC08_L1TP_018016_20180713_20180730_01_T1_B4.JPG',
 'LC08_L1TP_221076_20180727_20180731_01_T1_B4.JPG']

In [36]:
#os.chdir('/root/contrail_processed/Neg/filename00012_archive')
os.chdir('/root/contrail_processed/Pos/')

In [37]:
from IPython.core.display import HTML
for filename in jpg_files:  #contrail_processed/Pos/LC08_L1TP_020026_20180217_20180307_01_T1_B10.JPG
    with open(filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
    #        f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
#            f'<img src=filename alt=filename align="left" style="width: 250px;"/>'     
#            f"<figcaption>{filename} Predicted Label: {predicted_label}</figcaption>"
            f"<figcaption>{filename} Predicted Label: {predicted_label} Model Predictions: {model_predictions}</figcaption>"
        )
    )
    query_response

In [45]:
os.chdir('/root/contrail_processed/Pos')

In [ ]:
jpg_files

In [52]:
model_predictions

{'probabilities': [0.0869820938, 0.913017929],
 'labels': ['neg', 'pos'],
 'predicted_label': 'pos'}

---
Next, we clean up the deployed endpoint.

---

In [ ]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

## 4.7. Incrementally train the fine-tuned model

***
Incremental training allows you to train a new model using an expanded dataset that contains an underlying pattern that was not accounted for in the previous training and which resulted in poor model performance. You can use the artifacts from an existing model and use an expanded dataset to train a new model. Incremental training saves both time and resources as you don’t need to retrain a model from scratch.

One may use any dataset (old or new) as long as the dataset format remain the same (set of classes). Incremental training step is similar to the finetuning step discussed above with the following difference: In fine-tuning above, we start with a pre-trained model whereas in incremental training, we start with an existing fine-tuned model.
***

In [ ]:
# Identify the previously trained model path based on the output location where artifacts are stored previously and the training job name.


if use_amt:  # If using amt, select the model for the best training job.
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hp_tuner._current_job_name
    )
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name

last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"

In [ ]:
incremental_train_output_prefix = "jumpstart-example-ic-incremental-training"

incremental_s3_output_location = f"s3://{output_bucket}/{incremental_train_output_prefix}/output"

incremental_training_job_name = name_from_base(f"jumpstart-example-{model_id}-incremental-training")

incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=incremental_s3_output_location,
    base_job_name=incremental_training_job_name,
    metric_definitions=training_metric_definitions,
)

incremental_train_estimator.fit({"training": training_dataset_s3_path}, logs=True)

Once trained, we can use the same steps as in [Deploy & run Inference on the fine-tuned model](#4.5.-Deploy-&-run-Inference-on-the-fine-tuned-model) to deploy the model.